# Orquestrador de Chatbots - Etapa 2
-----------------------------------
## Limpeza, Vocabulário e Vetorização

1. Limpeza dos Dados
2. Construção do Vocabulário
3. Geração do vetorizador
4. Pesistência dos dados processados, do vocabulário e do vetorizador

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Bibliotecas utilizadas

In [21]:
import pandas as pd
import numpy as np
import csv
import codecs
import os
import glob
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
import orquestrador_funcoes_gerais as ofg

## Configurações

In [23]:
cfg = ofg.carregar_configuracoes()

Conferir as configurações antes de prosseguir
nome_arquivo_configuracao: config.json
------------------------------------------------------------------------------------------------------------------------
aplicar_stemmer: False
------------------------------------------------------------------------------------------------------------------------
bots: [{'bot_id': 1, 'nome': 'Alistamento Militar', 'arquivo': 'skill-alistamento-militar.json'}, {'bot_id': 2, 'nome': 'COVID', 'arquivo': 'skill-covid.json'}, {'bot_id': 3, 'nome': 'Login Único', 'arquivo': 'skill-login-unico.json'}, {'bot_id': 4, 'nome': 'IRPF 2020', 'arquivo': 'skill-perguntao-irpf-2020.json'}, {'bot_id': 5, 'nome': 'PGMEI', 'arquivo': 'skill-pgmei.json'}, {'bot_id': 6, 'nome': 'Selo Turismo Responsável', 'arquivo': 'skill-poc-selo-turismo-responsavel.json'}, {'bot_id': 7, 'nome': 'Cadastur', 'arquivo': 'skill-cadastur.json'}, {'bot_id': 8, 'nome': 'Tuberculose', 'arquivo': 'skill-tuberculose.json'}]
---------------------

In [24]:
bots=cfg['bots']

In [39]:
print('ATENÇÃO!!! Aplicação de Stemmer =', cfg['aplicar_stemmer'])

ATENÇÃO!!! Aplicação de Stemmer = False


## Carregar e preparar os dados

In [26]:
# Carrega o arquivo CSV com as perguntas de treino e teste
arquivo_treino_testes = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_treino_testes']) 
df = pd.read_csv(arquivo_treino_testes, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df), 'de', cfg['arquivo_treino_testes'])

# Exibe uma amostra dos dados carregados
df

Total de registros carregados: 2650 de treino_testes.csv


,bot_id,pergunta
0,1,tenho MAIS DE 18 devo me alistar?
1,8,aperto de mao transmite tuberculose
2,6,A DIÁRIA TERÁ UM VALOR MAIOR para quem aderir ...
3,3,preciso de uma conta de acesso no login unico
4,3,Como resolver problema de CPF inválido ?
...,...,...
2645,4,filho como dependente
2646,4,o contribuinte esta obrigado ao preenchimento ...
2647,5,Preciso imprimir a guia MEI em atraso. Como faço?
2648,7,o quer dizer CNAE


In [27]:
# Distribuição das classes nos dados fornecidos. Note que não há nenhum pergunta rotulada como "0" no arquivo de treino e testes
# Há um grande desbalanceamento das classes. Futuramente vale considerar métodos mais sofisticados de balanceamento.
df.bot_id.value_counts()

4    1202
6     389
3     292
1     237
7     191
8     151
2     116
5      72
Name: bot_id, dtype: int64

In [28]:
# Processa as perguntas, fazendo a limpeza dos textos
df['pergunta'] = df['pergunta'].apply(lambda x: ofg.limpar_texto(x, cfg['aplicar_stemmer']))
df

,bot_id,pergunta
0,1,dezoito devo alistar
1,8,aperto mao transmite tuberculose
2,6,diaria tera valor maior aderir selo turismo re...
3,3,preciso conta acesso login unico
4,3,resolver problema cpf invalido
...,...,...
2645,4,filho dependente
2646,4,contribuinte obrigado preenchimento numero recibo
2647,5,preciso imprimir guia microempreendedor indivi...
2648,7,quer dizer cnae


## Gerar Vocabulário

In [29]:
df["tokens"] = df["pergunta"].apply(ofg.tokenizer.tokenize)
freq_doc = dict()
vocab = dict()

print("\nINÍCIO CONSTRUÇÃO VOCABULÁRIO.")
for index in range(len(df)):
    tokens = df.at[index,'tokens']
    for token in tokens:
        if token in vocab:
            vocab[token] += 1
        else:
            vocab[token] = 1
    for token in set(tokens):
        if token in freq_doc:
            freq_doc[token] += 1
        else:
            freq_doc[token] = 1
print("\nFIM CONSTRUÇÃO VOCABULÁRIO.\n")

vocabulario = list(vocab.keys())
vocabulario.sort()
print('Tamanho Vocabulário:',len(vocabulario))



INÍCIO CONSTRUÇÃO VOCABULÁRIO.

FIM CONSTRUÇÃO VOCABULÁRIO.

Tamanho Vocabulário: 2135


In [30]:
# Removendo termos infrequentes.
if cfg['vocab']['freq_min'] > 1:
    print('Removendo termos infrequentes...')
    vocab_df = pd.DataFrame({'palavra': list(vocab.keys()), 'frequencia': list(vocab.values())})
    vocab_remover = list(vocab_df[vocab_df['frequencia'] < cfg['vocab']['freq_min']]['palavra'])
    vocabulario = [p for p in vocabulario if p not in vocab_remover]
    print("Tamanho do vocabulário após remoção de palavras infrequentes: " + str(len(vocabulario)))

In [31]:
# Remoção das N palavras mais presentes em documentos.
if cfg['vocab']['remover_frequentes'] > 0:
    freq_doc_df = pd.DataFrame({'palavra': list(freq_doc.keys()), 'freq_docs': list(freq_doc.values())})
    palavras_freq_doc = freq_doc_df.sort_values(by="freq_docs", ascending=False)[0:cfg['vocab']['remover_frequentes']]    

    vocab_remover = list(palavras_freq_doc['palavra'])
    vocabulario = [p for p in vocabulario if p not in vocab_remover]
    print("Tamanho do vocabulário após remoção de palavras muito frequentes: " + str(len(vocabulario)))

In [32]:
if False:
    # Adequando texto ao vocabulário
    for index in range(len(df)):
        tokens = ofg.tokenizer.tokenize(df.at[index,'pergunta'])
        tokens = [palavra for palavra in tokens if palavra in vocabulario]
        df.at[index,'pergunta'] = ' '.join(tokens)
    print('Textos foram adaptados ao vocabulário')    

## Salvar Vocabulário e Textos Processados

In [33]:
# Antes de salvar, elimina perguntas abaixo de um certo tamanho
mask = (df['pergunta'].str.len() > 4)
df = df.loc[mask]

In [34]:
# Salva textos processados
arquivo_treino_testes_processado = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_treino_testes_processado'])
df.to_csv(arquivo_treino_testes_processado, index=False, columns=['bot_id', 'pergunta'])
print('Textos processados salvos em',arquivo_treino_testes_processado)

Textos processados salvos em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\treino_testes_processado.csv


In [35]:
# Salva vocabulário
arquivo_vocab = os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_vocabulario'])

txt = ''
for palavra in vocabulario:
    txt += palavra + '\n'
f = open(arquivo_vocab, 'w')
f.write(txt)
f.close()
print('Vocabulário salvo em', arquivo_vocab)

Vocabulário salvo em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\vocabulario.txt


## Criar e Salvar Vetorizador

In [36]:
cfg_vect = cfg['vetorizador']

if cfg_vect['vector_vocab']:
    vectorizer = TfidfVectorizer(ngram_range=cfg_vect['ngram_range'], vocabulary=vocabulario)
else:
    vectorizer = TfidfVectorizer(ngram_range=cfg_vect['ngram_range'], sublinear_tf=cfg_vect['sublinear_tf'], 
                                 smooth_idf=cfg_vect['smooth_idf'], vocabulary=None, norm=cfg_vect['norm'])
    
vectorizer.fit_transform(df['pergunta'].tolist())
print(vectorizer)

TfidfVectorizer(ngram_range=[1, 2], smooth_idf=False, sublinear_tf=True)


In [37]:
# Persistindo o vetorizador
arquivo_vetorizador = os.path.join(os.getcwd(), cfg['diretorio_modelos'], cfg['arquivo_vetorizador'])

with open(arquivo_vetorizador, "wb") as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
print('Vetorizador persistido em', arquivo_vetorizador)

Vetorizador persistido em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\vetorizador.pkl


In [38]:
print('Fim da etapa 2!')

Fim da etapa 2!
